# Peer-graded Assignment: The Battle of Neighborhoods - Part 1

## Objective: define good location for a new business

## 1) Description of the problem and a discussion of the background

Frankfurt at Main is known as a very international city in Germany. The developed bank and finance sectors continuously attract skilled workers from abroad. As a consequence, demands on other sectors such as hotels, properties, restaurants, entertainment, and transportation are also increasing and prices are skyrocketing as a consequence. 

To support these changes, the city is opening credit lines for local entrepreneurs to invest in these sectors and enjoy this economical growth.

Observing a good opportunity and knowing that the German cuisine is not quite appealing for foreigners, a skilled Brazilian chef de cuisine decided to apply for the available credit line and also invest all his savings in opening a Philippine restaurant. 

However, the soon to be a restaurant owner is aware of the challenges ahead and given that I) the target clients are very demanding, II) the restaurant location has to be close to the finance and banking area, and III) that the prices of renting in the city are soaring, the business location is crucial for its long-term success.

#### The chef de cuisine hired a data scientist to locate good place for the business. The restaurant has to be within 2 km from the center of the financing neighborhood, close to popular attractions, and avoid being close to other Asian restaurants. 

## 2) Description of the data and how it will be used to solve the problem

The target area for new the business location is Frankfurt's banking and financial neighborhood.

The analysis will require Foursquare API to retrieve information about the location, rating, description, and popularity of restaurants and attractions in the area.

#### After obtaining and cleaning the dataset, we are going to apply clustering/classification models to determine hotspots for the restaurant implementation.

#### Some sneak preview of the data:

In [2]:
import requests
import pandas as pd
from pandas import json_normalize

# Frankfurt's finance center lat / lon
fkf_latlon=[50.110924,8.682127]

# Foursquare API credentials are stored in a hidden file
from secret_settings import * 

# Foursquare API request parameters
LIMIT = 100 
radius = 2000

# request from Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(    
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    fkf_latlon[0], 
    fkf_latlon[1], 
    radius, 
    LIMIT)

raw_data = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = raw_data['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Kleinmarkthalle,Market,50.112778,8.682958
1,Weinterasse Rollanderhof,Wine Bar,50.112473,8.682164
2,Römerberg,Plaza,50.110489,8.682131
3,SCHIRN Kunsthalle,Art Museum,50.110291,8.683542
4,Hoppenworth & Ploch,Café,50.110891,8.683701


## 3) Quick preview on the city

In [3]:
import folium
# create map with activities
city_map = folium.Map(location=[fkf_latlon[0],fkf_latlon[1]], zoom_start=12)
folium.Circle([fkf_latlon[0],fkf_latlon[1]],
                    radius=2000
                   ).add_to(city_map)  
# call the map
city_map